In [30]:
# run stopwatch
from _classes import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load Saved Section if exists

### Load libraries, functions, palette, theme

In [31]:
%run _load.ipynb

In [32]:
economics_data = loadit(
    'economics_data', 'data/00-Datasets/'
)

In [33]:
dir = '009.4-009.8_PI-Consumers-Indexes'

In [34]:
save_df = 'data/00-Datasets/'

In [35]:
save_excel = 'data/00-Excel-data/'

# <font color='#2C8B6D'>II. Цены, инфляция</font>

# Section I. Потребительские цены

## 9. Индексы потребительских цен

### 9.4. Индексы потребительских цен на товары и услуги по Российской Федерации, федеральным округам и субъектам Российской Федерации (с 1992 г.)

*Источник: https://rosstat.gov.ru/statistics/price*  
*Единицы измерения: *

*Примечания:*  

In [36]:
pci_link = \
    'https://rosstat.gov.ru/storage/mediabank/ipc_s_1992.xlsx'

In [37]:
pci_raw = pd.read_excel(pci_link)

In [38]:
pci_raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,ИНДЕКСЫ ПОТРЕБИТЕЛЬСКИХ ЦЕН\nна все товары и у...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"на конец периода, в % к декабрю предыдущего года",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1992,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,...,2013.0,2014.0,2015.0,2016,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,4) до 2008 г. Иркутская область и Усть-Ордынск...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,5) до 2008 г. Читинская область и Агинский Бу...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,6) до 2007 г. Камчатская область и Корякский АО.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
pci = pci_raw.iloc[2:-7, :].copy()

In [40]:
pci.iloc[0, 0] = 'Регион'

In [41]:
pci.columns = pci.iloc[0, :]
pci = pci.drop(2, axis=0)

In [42]:
pci.columns = [to_int(i) for i in pci.columns]
pci.columns.name = None

ValueError: Регион


In [43]:
pci['Регион'] = [i.strip() for i in pci['Регион']]

In [44]:
replace_list = ['1)', '2)', '3)', '4)', '5)', '6)']

In [45]:
pci['Регион'] = [replace_elements(i, '', replace_list) for i in pci['Регион']]

In [46]:
astrakhan_names = [
    'Архангельская область (кроме Ненецкого автономного округа)', 'Архангельская область']
pci = sum_rows( pci, 'Регион', astrakhan_names, 'Архангельская область без автономных округов')

In [47]:
tyumen_names = [
    'Тюменская область (кроме Ханты-Мансийского автономного округа - Югры и Ямало-Ненецкого автономного округа)',
    'Тюменская область']
pci = sum_rows(pci, 'Регион', tyumen_names, 'Тюменская область без автономных округов')

In [48]:
pci = pci[~pci['Регион'].isin(federal_districts_names_list)].copy()

In [49]:
pci = pci[~pci['Регион'].isin(regions_drop_list)].copy()

In [50]:
pci['Регион'] = pci['Регион'].replace(regions_replace_dict, regex=True)
pci['Регион'] = pci['Регион'].replace(regions_replace_dict)

In [51]:
pci = pci.T.copy()

In [52]:
pci.columns = pci.loc['Регион']
pci = pci.drop('Регион', axis=0)
pci.columns.name = None

In [53]:
pci = pci.astype(float)

In [54]:
pci = transform_round(pci, 1)

In [55]:
isna(pci.T)

,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Республика Крым,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,142.5,127.6,107.2,101.4,105.5,102.6,105.2,108.7,113.2
Севастополь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,136.1,121.0,106.5,103.8,105.1,102.5,105.6,110.1,112.1
Республика Ингушетия,NaN,NaN,337.2,196.0,117.4,106.9,172.8,152.3,122.7,138.7,...,104.5,108.1,117.5,102.6,104.0,101.9,103.4,105.8,108.9,118.6
Чеченская Республика,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.2,111.1,114.8,105.2,102.0,103.5,103.0,105.0,107.6,112.9


In [56]:
pci.head()

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха,Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская авт.область,Чукотский авт.округ
1992,2608.8,2601.9,1758.8,2349.4,2584.7,2780.9,2258.7,2866.8,2425.8,1998.0,...,2627.6,3115.4,4896.5,2883.9,3063.1,2654.0,5265.0,3185.8,2138.6,2551.0
1993,939.9,791.9,810.7,912.4,824.3,829.3,874.9,828.2,960.3,870.6,...,1008.8,1359.3,1643.0,1130.0,1123.3,1203.5,1163.0,1718.4,1520.0,2058.0
1994,315.1,316.4,325.6,313.7,300.6,331.5,336.2,323.1,344.1,319.5,...,257.7,305.2,305.5,309.9,306.4,291.9,249.0,365.2,318.6,343.0
1995,231.3,228.4,261.4,240.2,240.4,238.3,256.9,245.1,251.1,256.3,...,232.5,236.8,239.2,222.3,218.2,218.4,200.4,239.0,244.6,247.0
1996,121.8,129.2,119.2,117.5,129.8,124.4,120.5,121.0,120.0,122.6,...,121.8,121.1,127.3,122.2,120.6,122.1,121.3,131.8,125.5,156.0


### Индексы потребительских цен на товары и услуги по Российской Федерации, месяцы (с 1991 г.)

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: *

*Примечания:*  

In [57]:
pci_month_link = \
    'https://rosstat.gov.ru/storage/mediabank/ipc_mes-12.xlsx'

### 9.5. Индексы потребительских цен на <font color='#2C8B6D'> товары и услуги </font> по Российской Федерации в 1991-2023 гг.

In [58]:
pci_month_raw = pd.read_excel(pci_month_link, sheet_name='01')

In [59]:
pci_month_raw

,Индексы потребительских цен на товары и услуги1) по Российской Федерации в 1991-2023*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,К содержанию,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021,2022.0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,1)в соответствии с Федеральным планом ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,*)Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
pci_month = pci_month_raw[2:-3].copy()

In [61]:
pci_month.iloc[0,0] = '-'

In [62]:
pci_month.columns = pci_month.iloc[0, :]

In [63]:
pci_month = pci_month[2:].copy()

In [64]:
pci_month.columns = [to_int(i) for i in pci_month.columns]

ValueError: -


In [65]:
pci_month = pci_month.drop(16, axis=0)

In [66]:
pci_month.loc[17, '-'] = 'Год'

In [67]:
pci_month = pci_month.set_index('-', drop=True)

In [68]:
pci_month.index.name = None

In [69]:
replace_dict = {'1)': '', '2)': '', ',': '.'}

In [70]:
pci_month = pci_month.astype(str)

In [71]:
for column in pci_month.columns:
    for key, value in replace_dict.items():
        pci_month[column] = pci_month[column].str.replace(key, value)

In [72]:
pci_month = pci_month.astype(float)

In [73]:
pci_month = transform_round(pci_month, 1)

In [74]:
isna(pci_month)

There're no NaN values in DataFrame


In [75]:
# extract last row with 'Год' from 'pci_month'
first_row = pci_month.iloc[-1].to_frame().T

In [76]:
# add it as first row to 'pci_month'
pci_month = pd.concat([first_row, pci_month], axis=0)

In [77]:
# remove last row from 'pci_month'
pci_month = pci_month.iloc[:-1]

In [78]:
pci_month_rus = pci_month.copy()

In [79]:
pci_month_rus

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Год,260.4,2608.8,939.9,315.1,231.3,121.8,111.0,184.4,136.5,120.2,...,111.4,112.9,105.4,102.5,104.3,103.0,104.9,108.4,111.9,107.4
январь,106.2,345.3,125.8,117.9,117.8,104.1,102.3,101.5,108.4,102.3,...,100.6,103.8,101.0,100.6,100.3,101.0,100.4,100.7,101.0,100.8
февраль,104.8,138.0,124.7,110.8,111.0,102.8,101.5,100.9,104.1,101.0,...,100.7,102.2,100.6,100.2,100.2,100.4,100.3,100.8,101.2,100.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
октябрь,103.5,122.9,119.5,115.0,104.7,101.2,100.2,104.5,101.4,102.1,...,100.8,100.7,100.4,100.2,100.4,100.1,100.4,101.1,100.2,100.8
ноябрь,108.9,126.1,116.4,114.6,104.6,101.9,100.6,105.7,101.2,101.5,...,101.3,100.8,100.4,100.2,100.5,100.3,100.7,101.0,100.4,101.1
декабрь,112.1,125.2,112.5,116.4,103.2,101.4,101.0,111.6,101.3,101.6,...,102.6,100.8,100.4,100.4,100.8,100.4,100.8,100.8,100.8,100.7


In [111]:
with pd.option_context('display.max_rows', 15):
    display(pci_month_rus)

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Год,260.4,2608.8,939.9,315.1,231.3,121.8,111.0,184.4,136.5,120.2,...,111.4,112.9,105.4,102.5,104.3,103.0,104.9,108.4,111.9,107.4
январь,106.2,345.3,125.8,117.9,117.8,104.1,102.3,101.5,108.4,102.3,...,100.6,103.8,101.0,100.6,100.3,101.0,100.4,100.7,101.0,100.8
февраль,104.8,138.0,124.7,110.8,111.0,102.8,101.5,100.9,104.1,101.0,...,100.7,102.2,100.6,100.2,100.2,100.4,100.3,100.8,101.2,100.5
март,106.3,129.9,120.1,107.4,108.9,102.8,101.4,100.6,102.8,100.6,...,101.0,101.2,100.5,100.1,100.3,100.3,100.6,100.7,107.6,100.4
апрель,163.5,121.7,118.7,108.5,108.5,102.2,101.0,100.4,103.0,100.9,...,100.9,100.5,100.4,100.3,100.4,100.3,100.8,100.6,101.6,100.4
май,103.0,111.9,118.1,106.9,107.9,101.6,100.9,100.5,102.2,101.8,...,100.9,100.4,100.4,100.4,100.4,100.3,100.3,100.7,100.1,100.3
июнь,101.2,119.1,119.9,106.0,106.7,101.2,101.1,100.1,101.9,102.6,...,100.6,100.2,100.4,100.6,100.5,100.0,100.2,100.7,99.6,100.4
июль,100.6,110.6,122.4,105.3,105.4,100.7,100.9,100.2,102.8,101.8,...,100.5,100.8,100.5,100.1,100.3,100.2,100.4,100.3,99.6,100.6
август,100.5,108.6,126.0,104.6,104.6,99.8,99.9,103.7,101.2,101.0,...,100.2,100.4,100.0,99.5,100.0,99.8,100.0,100.2,99.5,100.3
сентябрь,101.1,111.5,123.0,108.0,104.5,100.3,99.7,138.4,101.5,101.3,...,100.6,100.6,100.2,99.8,100.2,99.8,99.9,100.6,100.0,100.9


In [80]:
pci_month.index = ['Year'] + [months_translate(x) for x in pci_month.index[1:]]

In [81]:
pci_month

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Year,260.4,2608.8,939.9,315.1,231.3,121.8,111.0,184.4,136.5,120.2,...,111.4,112.9,105.4,102.5,104.3,103.0,104.9,108.4,111.9,107.4
January,106.2,345.3,125.8,117.9,117.8,104.1,102.3,101.5,108.4,102.3,...,100.6,103.8,101.0,100.6,100.3,101.0,100.4,100.7,101.0,100.8
February,104.8,138.0,124.7,110.8,111.0,102.8,101.5,100.9,104.1,101.0,...,100.7,102.2,100.6,100.2,100.2,100.4,100.3,100.8,101.2,100.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
October,103.5,122.9,119.5,115.0,104.7,101.2,100.2,104.5,101.4,102.1,...,100.8,100.7,100.4,100.2,100.4,100.1,100.4,101.1,100.2,100.8
November,108.9,126.1,116.4,114.6,104.6,101.9,100.6,105.7,101.2,101.5,...,101.3,100.8,100.4,100.2,100.5,100.3,100.7,101.0,100.4,101.1
December,112.1,125.2,112.5,116.4,103.2,101.4,101.0,111.6,101.3,101.6,...,102.6,100.8,100.4,100.4,100.8,100.4,100.8,100.8,100.8,100.7


### 9.6. Индексы потребительских цен на <font color='#2C8B6D'> продовольственные товары </font> по Российской Федерации в 1991-2023 гг.

In [82]:
pci_food_raw = pd.read_excel(pci_month_link, sheet_name='02')

In [83]:
pci_food_raw.tail(20)

,Индексы потребительских цен на продовольственные товары по Российской Федерации в 1991-2023*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021,2022.0,2023
3,к концу предыдущего месяца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,1)Декабрь 2023 г. в % к декабрю 2022 г.\n Обра...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,*)Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
pci_food = transform_pci(pci_food_raw)

ValueError: -


In [85]:
pci_food.index = ['Year'] + [months_translate(x) for x in pci_food.index[1:]]

In [86]:
pci_food

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Year,236.1,2626.2,904.9,314.1,223.4,117.7,109.1,196.0,135.9,117.9,...,115.4,114.0,104.6,101.1,104.7,102.6,106.7,110.6,110.3,108.2
January,104.6,405.1,129.0,121.0,121.0,104.0,103.0,102.1,110.3,102.2,...,101.0,105.7,101.2,100.8,100.5,101.3,100.7,101.0,101.4,101.3
February,103.1,129.8,124.8,110.6,110.3,102.4,101.4,101.2,104.4,100.5,...,101.2,103.3,100.7,100.2,100.4,100.8,100.6,101.2,101.6,100.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
October,102.3,124.7,118.5,115.3,103.4,100.4,99.5,103.9,100.9,102.1,...,101.2,101.0,100.8,100.4,100.6,100.2,100.6,102.2,100.3,101.4
November,110.6,127.3,115.9,117.3,104.0,102.2,100.4,107.6,101.0,101.5,...,102.0,101.2,100.7,100.2,101.0,100.4,101.3,101.3,100.4,101.6
December,116.1,125.6,112.5,120.9,103.4,102.0,101.2,117.0,101.4,101.9,...,103.2,101.2,100.6,100.6,101.7,100.6,101.5,101.4,100.6,101.5


### 9.7. Индексы потребительских цен на <font color='#AF4035'> непродовольственные товары </font>  по Российской Федерации в 1991-2023гг.

In [87]:
pci_goods_raw = pd.read_excel(pci_month_link, sheet_name='03')

In [88]:
pci_goods_raw.head(20)

,Индексы потребительских цен на непродовольственные товары по Российской Федерации в 1991-2023*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,К содержанию,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.00,1993.0,1994.0,1995.0,1996.00,1997.00,1998.00,1999.00,...,2014.00,2015.00,2016.00,2017.00,2018.0,2019.00,2020.00,2021,2022.0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,декабрь,310.7,2673.44,741.8,269.0,216.3,117.82,108.11,199.54,139.15,...,108.05,113.65,106.54,102.75,104.1,102.95,104.79,108.58,112.7,"105,961)"
18,1)Декабрь 2023 г. в % к декабрю 2022 г.\n Обра...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
pci_goods = transform_pci(pci_goods_raw)

ValueError: -


In [90]:
pci_goods.index = ['Year'] + [months_translate(x) for x in pci_goods.index[1:]]

In [91]:
pci_goods

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Year,310.7,2673.4,741.8,269.0,216.3,117.8,108.1,199.5,139.2,118.5,...,108.0,113.6,106.5,102.8,104.1,103.0,104.8,108.6,112.7,106.0
January,109.0,311.1,116.3,110.7,112.0,102.7,101.0,100.5,106.2,102.2,...,100.3,103.2,100.7,100.5,100.3,100.6,100.2,100.5,100.7,100.2
February,105.9,147.6,122.8,108.9,109.8,102.2,100.6,100.3,104.0,101.3,...,100.4,102.1,100.8,100.2,100.1,100.3,100.0,100.6,100.8,99.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
October,105.0,119.9,119.3,116.4,105.0,101.2,100.8,107.4,102.2,101.9,...,100.6,101.0,100.5,100.3,100.4,100.3,100.7,100.8,100.0,100.6
November,108.6,125.6,115.6,112.5,104.6,100.9,100.7,104.3,101.5,101.5,...,100.6,100.7,100.4,100.3,100.4,100.2,100.6,100.7,100.1,100.5
December,110.5,126.9,111.6,111.9,103.0,100.7,100.6,106.3,101.1,101.2,...,102.2,100.4,100.3,100.3,100.2,100.1,100.4,100.6,100.0,100.4


### 9.8. Индексы потребительских цен на <font color='#5081AA'> услуги </font>  по Российской Федерации в 1991-2023гг.

In [92]:
pci_services_raw = pd.read_excel(pci_month_link, sheet_name='04')

In [93]:
pci_services_raw.head(20)

,Индексы потребительских цен на услуги по Российской Федерации в 1991-2023*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,К содержанию,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.00,1993.0,1994.0,1995.00,1996.00,1997.00,1998.00,1999.00,...,2014.00,2015.0,2016.00,2017.00,2018.00,2019.00,2020.0,2021,2022.00,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,декабрь,178.8,2220.52,2411.2,622.4,332.21,148.38,122.52,118.29,134.02,...,110.45,110.2,104.89,104.35,103.94,103.75,102.7,104.98,113.19,"108,331)"
18,1)Декабрь 2023 г. в % к декабрю 2022 г.\n Обра...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
pci_services = transform_pci(pci_services_raw)

ValueError: -


In [95]:
pci_services.index = ['Year'] + [months_translate(x) for x in pci_services.index[1:]]

In [96]:
pci_services

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Year,178.8,2220.5,2411.2,622.4,332.2,148.4,122.5,118.3,134.0,133.7,...,110.4,110.2,104.9,104.4,103.9,103.8,102.7,105.0,113.2,108.3
January,103.2,230.0,150.0,137.7,122.7,108.0,102.3,101.6,104.1,103.4,...,100.5,102.2,101.0,100.5,100.1,101.1,100.2,100.4,100.8,101.0
February,105.2,140.0,132.0,121.5,119.3,105.6,103.6,101.0,103.2,103.0,...,100.4,100.8,100.3,100.3,100.1,100.2,100.4,100.4,101.1,100.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
October,102.6,128.9,124.3,110.5,108.9,103.9,101.2,101.6,102.0,102.4,...,100.6,99.9,99.7,99.8,99.9,99.8,99.9,100.0,100.2,100.5
November,103.9,122.6,123.0,110.5,106.6,102.7,101.1,101.2,101.7,101.6,...,101.2,100.2,100.0,100.1,100.0,100.1,100.0,100.8,100.8,101.2
December,103.4,116.2,115.6,111.5,103.0,100.8,100.7,101.8,100.9,101.6,...,102.2,100.6,100.3,100.3,100.4,100.2,100.4,100.3,102.0,100.1


##### <font color='#820A22'> Export Dataset to Excel

In [97]:
datasets_list = [
    pci, pci_month, pci_food, pci_goods, pci_services
]

In [98]:
sheet_names_list = [
    'ИПЦ (регионы 1992-2022)',
    'ИПЦ Товары и услуги (мес.)', 'ИПЦ Питание (мес.)',
    'ИПЦ Товары (мес.)', 'ИПЦ Услуги (мес.)'
]

In [99]:
for dataset, sheet in zip(datasets_list, sheet_names_list):
    saveit_excel(
    data=dataset,
    filename=dir,
    path=save_excel,
    sheet=sheet
)

'ИПЦ (регионы 1992-2022)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Товары и услуги (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Питание (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Товары (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Услуги (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'


In [100]:
saveit_excel(
    data=pci_month_rus,
    filename=dir+'_RUS',
    path=save_excel,
    sheet='ИПЦ (регионы 1992-2022)'
)

'ИПЦ (регионы 1992-2022)' sheet created if file '009.4-009.8_PI-Consumers-Indexes_RUS.xlsx'


### Save Dataset

In [101]:
economics_data['9.4'] = pci

In [102]:
economics_data['9.5'] = pci_month

In [103]:
economics_data['9.6'] = pci_food

In [104]:
economics_data['9.7'] = pci_goods

In [105]:
economics_data['9.8'] = pci_services

In [106]:
saveit(
    file=economics_data,
    name='economics_data',
    dir=save_df
)

File 'economics_data' saved in directory 'data/00-Datasets/'


### Save Session

In [107]:
save_session(dir)

### Execution time

In [108]:
print(f'Execution time: {stopwatch.stop(start)}')

Execution time: 0:00:16
